In [1]:
import numpy as np
import cv2

In [3]:
path = './cat.jpg'

# Загружаем изображение
img = cv2.imread(path)

# Проверяем, что изображение не менее 1000x800 пикселей
if img.shape[0] < 1000 or img.shape[1] < 800:
    print("Изображение должно быть не менее 1000x800 пикселей.")

# Изменение формы изображения аффинным преобразованием
M = np.float32([[1, 0.5], [0, 1]])
img_affine = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

# Изменение формы изображения проективным преобразованием
M = np.float32([[1, 0.5], [0, 1.5]])
img_projective = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))

# Применение деформации к изображению
map_x, map_y = np.meshgrid(np.arange(img.shape[1]), np.arange(img.shape[0]))
map_x = map_x + np.sin(map_x / 50)
map_y = map_y + np.sin(map_y / 50)
img_warp = cv2.remap(img, map_x, map_y, cv2.INTER_LINEAR)

# Разделение изображения на четыре части
img_parts = np.array_split(img, 4, axis=1)
img_parts = [np.array_split(part, 4, axis=0) for part in img_parts]

# Обнуление каналов в одной части
img_parts[0][0][0] = img_parts[0][0][0]  *  0

# Переворот второй части
img_parts[1][0] = img_parts[1][0][::-1, :]

# Суммирование каналов в третьей части
img_parts[2][0] = img_parts[2][0] + img_parts[2][1] + img_parts[2][2]

# Вставка фрагментов из других частей в четвертую часть
img_parts[3][0][0] = img_parts[3][0][0] + img_parts[0][0][0]
img_parts[3][0][1] = img_parts[3][0][1] + img_parts[1][0][0]
img_parts[3][0][2] = img_parts[3][0][2] + img_parts[2][0][0]

# Объединение частей обратно в изображение
img_combined = np.hstack(np.vstack(img_parts))

# Вставка изображения в рамку
img_with_frame = cv2.copyMakeBorder(img_combined, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=(0, 0, 0))

# Рисование и подписание животного или выдуманного персонажа
img_with_animal = img.copy()
cv2.rectangle(img_with_animal, (100, 100), (300, 300), (0, 255, 0), 3)
cv2.putText(img_with_animal, 'Animal', (150, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

# Вставка изображения с животным на другое изображение
img_with_both = cv2.addWeighted(img_with_frame, 0.5, img_with_animal, 0.5, 0)

# Сохранение полученного изображения
cv2.imwrite('output.jpg', img_with_both)

error: OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\imgproc\src\imgwarp.cpp:2619: error: (-215:Assertion failed) (M0.type() == 5 || M0.type() == 6) && M0.rows == 2 && M0.cols == 3 in function 'cv::warpAffine'
